In [65]:
using CSV
using DataFrames
using Printf 
using Statistics
using Plots
using Combinatorics
using GLM
using StatsBase
using Random

file_path = "C:/Users/Usuario/Desktop/my_dataframe.csv"  
data = CSV.read(file_path, DataFrame)

println("Primeras 10 filas del dataset:")
println(first(data, 5))

Primeras 10 filas del dataset:
5×5 DataFrame
 Row │ R_Depth  R_SALINITY  R_SIGMA  R_DYNHT  T_degC  
     │ Int64    Float64     Float64  Float64  Float64 
─────┼────────────────────────────────────────────────
   1 │       0      33.44     25.64     0.0     10.5
   2 │       8      33.44     25.65     0.01    10.46
   3 │      10      33.437    25.65     0.02    10.46
   4 │      19      33.42     25.64     0.04    10.45
   5 │      20      33.421    25.64     0.04    10.45


## Explorar los datos para identificar variables independientes y dependientes :

In [14]:
println(names(data))

println(dropmissing!(data))  


Excessive output truncated after 10485817 bytes.

["R_Depth", "R_SALINITY", "R_SIGMA", "R_DYNHT", "T_degC"]

In [12]:
print(describe(data))

6×7 DataFrame
 Row │ variable    mean        min     median   max      nmissing  eltype   
     │ Symbol      Float64     Real    Float64  Real     Int64     DataType 
─────┼──────────────────────────────────────────────────────────────────────
   1 │ R_Depth     170.718      0      123.0    681             0  Int64
   2 │ R_SALINITY   33.8219    28.431   33.84    37.034         0  Float64
   3 │ R_SIGMA      25.7443    20.934   25.928  250.784         0  Float64
   4 │ R_DYNHT       0.383188   0.0      0.33     1.52          0  Float64
   5 │ R_PRES      171.663      0      124.0    686             0  Int64
   6 │ T_degC       11.2059     3.65    10.32    31.14          0  Float64

In [115]:
dependent_var = :T_degC  # Variable dependiente
independent_vars = Symbol.(["R_Depth", "R_SALINITY", "R_SIGMA", "R_DYNHT"])  # Convertir nombres a símboloss


4-element Vector{Symbol}:
 :R_Depth
 :R_SALINITY
 :R_SIGMA
 :R_DYNHT

In [116]:
# Particionar el dataset
function partition_data(data::DataFrame, train_fraction::Float64)
    n = nrow(data)
    train_size = Int(round(train_fraction * n))
    indices = shuffle(1:n)
    train_indices = indices[1:train_size]
    test_indices = indices[train_size+1:end]
    return data[train_indices, :], data[test_indices, :]
end

partition_data (generic function with 1 method)

In [117]:
# Dividir en entrenamiento y prueba
train_data, test_data = partition_data(data, 0.7)

(481995×5 DataFrame
    Row │ R_Depth  R_SALINITY  R_SIGMA  R_DYNHT  T_degC  
        │ Int64    Float64     Float64  Float64  Float64 
────────┼────────────────────────────────────────────────
      1 │     300      34.094   26.62     0.62      7.6
      2 │     125      33.783   26.01     0.35      9.96
      3 │      75      33.506   25.45     0.23     11.88
      4 │      30      33.122   24.55     0.1      14.86
      5 │       0      32.92    24.8      0.0      12.87
      6 │      69      33.534   25.588    0.203    11.27
      7 │      10      34.41    22.92     0.04     24.92
      8 │     237      34.249   26.5      0.63      9.2
      9 │     250      34.128   26.57     0.52      8.13
     10 │     199      34.067   26.44     0.46      8.66
     11 │      20      33.41    23.98     0.08     18.28
   ⋮    │    ⋮         ⋮          ⋮        ⋮        ⋮
 481986 │     100      33.606   25.89     0.28      9.81
 481987 │     300      34.136   26.59     0.68      8.06
 481988 │    

In [118]:
names(train_data)

5-element Vector{String}:
 "R_Depth"
 "R_SALINITY"
 "R_SIGMA"
 "R_DYNHT"
 "T_degC"

In [125]:
# Definir una función para evaluar modelos
function evaluate_model(dependent_var::Symbol, independent_vars::Vector{Symbol}, train_data::DataFrame)
    formula = Term(dependent_var) ~ sum(Term(var) for var in independent_vars)
    model = lm(formula, train_data)
    return deviance(model), model
end

evaluate_model (generic function with 1 method)

In [126]:
# Probar combinaciones de variables independientes
best_model = nothing
best_deviance = Inf
best_combination = nothing

for k in 1:length(independent_vars)
    for combination in combinations(independent_vars, k)
        deviance, model = evaluate_model(dependent_var, combination, train_data)
        println("Combination: $combination, Deviance: $deviance")
        if deviance < best_deviance
            best_deviance = deviance
            best_model = model
            best_combination = combination
        end
    end
end


Combination: [:R_Depth], Deviance: 2.916607095440113e6
Combination: [:R_SALINITY], Deviance: 6.455858096922825e6
Combination: [:R_SIGMA], Deviance: 2.904590942447134e6
Combination: [:R_DYNHT], Deviance: 2.886236285863273e6
Combination: [:R_Depth, :R_SALINITY], Deviance: 2.7622332039154284e6
Combination: [:R_Depth, :R_SIGMA], Deviance: 2.0175754260623246e6
Combination: [:R_Depth, :R_DYNHT], Deviance: 2.8202954186916654e6
Combination: [:R_SALINITY, :R_SIGMA], Deviance: 2.901430269497727e6
Combination: [:R_SALINITY, :R_DYNHT], Deviance: 2.748660291300489e6
Combination: [:R_SIGMA, :R_DYNHT], Deviance: 2.017049041653835e6
Combination: [:R_Depth, :R_SALINITY, :R_SIGMA], Deviance: 1.7194300389269064e6
Combination: [:R_Depth, :R_SALINITY, :R_DYNHT], Deviance: 2.6488472105114185e6
Combination: [:R_Depth, :R_SIGMA, :R_DYNHT], Deviance: 1.9885837729358645e6
Combination: [:R_SALINITY, :R_SIGMA, :R_DYNHT], Deviance: 1.740853376826456e6
Combination: [:R_Depth, :R_SALINITY, :R_SIGMA, :R_DYNHT], Devia

In [131]:
function calculate_metrics(model, test_data, dependent_var)
    y_true = test_data[!, dependent_var]
    y_pred = predict(model, test_data)
    residuals = y_true - y_pred

    mse = mean(residuals .^ 2)  # Mean Squared Error
    rmse = sqrt(mse)  # Root Mean Squared Error
    mae = mean(abs.(residuals))  # Mean Absolute Error
    r_squared = 1 - (sum(residuals .^ 2) / sum((y_true .- mean(y_true)) .^ 2))  # R²

    return mse, rmse, mae, r_squared
end

calculate_metrics (generic function with 1 method)

In [132]:
# Calcular métricas en el conjunto de prueba
mse, rmse, mae, r_squared = calculate_metrics(best_model, test_data, dependent_var)

(2.2315867082840954, 1.4938496270656212, 1.2033166108423856, 0.8602037345774589)

In [134]:
# Mostrar resultados
println("\nMetrics on test data:")
println("MSE: $mse")
println("RMSE: $rmse")
println("MAE: $mae")
println("R²: $r_squared")


Metrics on test data:
MSE: 2.2315867082840954
RMSE: 1.4938496270656212
MAE: 1.2033166108423856
R²: 0.8602037345774589


In [135]:
# Mostrar el mejor modelo
println("\nBest combination: $best_combination")
println("Best deviance: $best_deviance")
println("Model summary:")
display(coeftable(best_model))


Best combination: [:R_Depth, :R_SALINITY, :R_SIGMA, :R_DYNHT]
Best deviance: 1.681783061923297e6
Model summary:


|             |        Coef. | Std. Error |       t | Pr(>|t|) |    Lower 95% |    Upper 95% |
|:------------|-------------:|-----------:|--------:|:---------|:-------------|-------------:|
| (Intercept) | -21.363      | 0.266946   |  -80.03 |   <1e-99 | -21.8862     | -20.8397 |
| R_Depth     |  -0.00866295 | 6.65805e-5 | -130.11 |   <1e-99 |  -0.00879345 |  -0.00853245 |
| R_SALINITY  |   2.35945    | 0.00795697 |  296.53 |   <1e-99 |   2.34385    |   2.37504 |
| R_SIGMA     |  -1.72057    | 0.00326821 | -526.46 |   <1e-99 |  -1.72697    |  -1.71416 |
| R_DYNHT     |  -3.79829    | 0.036567   | -103.87 |   <1e-99 |  -3.86996    |  -3.72662 |

In [136]:
display(coeftable(best_model))

|             |        Coef. | Std. Error |       t | Pr(>|t|) |    Lower 95% |    Upper 95% |
|:------------|-------------:|-----------:|--------:|:---------|:-------------|-------------:|
| (Intercept) | -21.363      | 0.266946   |  -80.03 |   <1e-99 | -21.8862     | -20.8397 |
| R_Depth     |  -0.00866295 | 6.65805e-5 | -130.11 |   <1e-99 |  -0.00879345 |  -0.00853245 |
| R_SALINITY  |   2.35945    | 0.00795697 |  296.53 |   <1e-99 |   2.34385    |   2.37504 |
| R_SIGMA     |  -1.72057    | 0.00326821 | -526.46 |   <1e-99 |  -1.72697    |  -1.71416 |
| R_DYNHT     |  -3.79829    | 0.036567   | -103.87 |   <1e-99 |  -3.86996    |  -3.72662 |